In [1]:
# McGreevey Final Project - BMIN 5200
# December 7, 2024
# The following is code for an expert system that provides clinical decision support 
# to physicians and other health care providers about how to use intravenous fluid (IVF)
# appropriately for a variety of common clinical conditions
# and according to the supply of IVF that that hospital has available



In [2]:
import clips
import sys
import logging
sys.path.append('./src/')
from clips_util import print_facts, print_rules, print_templates, build_read_assert

logging.basicConfig(level=10, format='%(message)s')

router = clips.LoggingRouter()
env = clips.Environment()
env.add_router(router)


In [3]:
#Patient age template
DEFTEMPLATE_PATIENT = """
(deftemplate patient 
    (slot name_is (type STRING))
    (slot age_is (type INTEGER))
)
"""
env.build(DEFTEMPLATE_PATIENT)

In [4]:
#Indication template
DEFTEMPLATE_INDICATION = """
(deftemplate indication
    (slot fluid_indication (type SYMBOL) (allowed-symbols shock rehydration maintenance contrast pancreatitis dka))
)
"""
env.build(DEFTEMPLATE_INDICATION)

In [5]:
#Fluid template
DEFTEMPLATE_FLUID = """
(deftemplate fluid
    (slot fluid_type (type SYMBOL) (allowed-symbols ns lr))
)
"""
env.build(DEFTEMPLATE_FLUID)

In [6]:
#IVF supply template
DEFTEMPLATE_SUPPLY = """
(deftemplate supply
    (slot supply_status (type SYMBOL) (allowed-symbols green yellow red))
)
"""
env.build(DEFTEMPLATE_SUPPLY)

In [7]:
#Heart failure template
DEFTEMPLATE_HF = """
(deftemplate hf
    (slot hf_status (type SYMBOL) (allowed-symbols yes no unknown))
)
"""
env.build(DEFTEMPLATE_HF)

In [8]:
# Add a deffact for initial hf_status to be set to 'unknown'
DEFFACTS_INITIAL_HF_STATUS = """
(deffacts initial_hf_status "Set the default value for HF to unknown"
    (hf (hf_status unknown))
)
"""
env.build(DEFFACTS_INITIAL_HF_STATUS)

# reset the environment to make sure the deffacts are added
#env.reset()

In [9]:
#Prompt map
prompt_map = {
    "patient:name_is": "Enter patient name: ",
    "patient:age_is": "Enter patient age (in years): ",
    "fluid:fluid_type": "Enter the desired IV fluid type (either lr or ns): ",
    "supply:supply_status": "What is the current IV fluid supply level? (green, yellow, or red): ",
    "indication:fluid_indication": "What is the indication for IV fluid? (shock rehydration maintenance contrast pancreatitis dka): ",
    "hf:hf_status": "Does this patient have a history of or suspicion for heart failure? (yes/no): "
}
build_read_assert(env, prompt_map)

In [10]:
#INITIAL QUERY DEFRULE
DEFRULE_INITIAL_QUERY = """
(defrule initial_query
    =>
    (read_assert patient)
    (read_assert indication)
    (read_assert supply)
)
"""
env.build(DEFRULE_INITIAL_QUERY)


In [11]:
#Rule to check patients >= 50 for possible HF, as HF may require more conservative fluid administration
#Limited to indications for which ivf recommendation would be adjusted for HF
DEFRULE_HF_READ = """
(defrule HF_READ ; does this patient have HF?
    (logical
    	(patient (age_is ?age))
    	(test(>= ?age 50))
    	(or 
            (indication (fluid_indication rehydration))
            (indication (fluid_indication maintenance))
            (indication (fluid_indication pancreatitis))
        )
 	)
    
    =>
   
    (read_assert hf)
)
"""
env.build(DEFRULE_HF_READ)


In [12]:
#Rule to prompt the fluid type question only for indications where clinicians have discretion in choosing fluid type
DEFRULE_ASK_FLUID= """
(defrule ask_fluid ; prompt fluid choice only if appropriate indication selected
    (or
        (indication (fluid_indication shock))
        (indication (fluid_indication rehydration))
        (indication (fluid_indication maintenance))
        (indication (fluid_indication pancreatitis))
        (indication (fluid_indication dka))
    )
    =>
    (println "Given the indication selected, you may choose a fluid type.")
    (read_assert fluid)
)
"""    
env.build(DEFRULE_ASK_FLUID)

In [13]:
#Defrules for IVF therapy recommendations begin here...

In [14]:
#Defule for green/yellow/shock #1
DEFRULE_GREEN_YELLOW_SHOCK = """
(defrule green_ns_shock "Rule to assess if green/yellow/shock criteria met"
    (logical
        (patient (name_is ?name))
        (patient (age_is ?age))
        (supply (supply_status ?supply))
        (fluid (fluid_type ?fluid))
        (indication (fluid_indication ?indication))
        
        (and  
            (indication (fluid_indication shock)) ; fluid indication is shock
            (or
                (supply (supply_status green)) ; green supply status
                (supply (supply_status yellow)); yellow supply status
            )
            
        )
    )    
    

    => 

    (println "___________")
    (println "For " ?name " for condition " ?indication " recommend 30ml/kg lr or ns bolus")
    (println "**lr is preferred**")
    (println "___________")
)
"""

env.build(DEFRULE_GREEN_YELLOW_SHOCK)

In [15]:
#Defule for red/shock #2
DEFRULE_RED_SHOCK = """
(defrule red_shock "Rule to assess if red/shock criteria met"
    (logical
        (patient (name_is ?name))
        (supply (supply_status ?supply))
        (fluid (fluid_type ?fluid))
        (indication (fluid_indication ?indication))
  
        (and  
            (supply (supply_status red)) ; green supply status
            (indication (fluid_indication shock)) ; fluid indication is shock  
                              
        )
    )    
    
 =>

   
    (println "___________")
    (println "For " ?name " for condition " ?indication " recommend considering vasopressor therapy. Consider 15-30ml/kg lr or ns bolus.\n **lr preferred**")
    (println "___________")
)
"""

env.build(DEFRULE_RED_SHOCK)

In [16]:
#Defule for green/rehydration #3
DEFRULE_GREEN_REHYDRATION = """
(defrule green_rehydration "Rule to assess if green/rehydration criteria met"
    (logical
        (patient (name_is ?name))
        (supply (supply_status ?supply))
        (fluid (fluid_type ?fluid))
        (indication (fluid_indication ?indication))
  
        (and  
            (supply (supply_status green)) ; green supply status
            (indication (fluid_indication rehydration)) ; fluid indication is rehydration  
            (or
                (hf (hf_status no)) ;hf is no
                (hf (hf_status unknown)) ;hf is unknown
            )                  
        )
    )    
    
 =>

   
    (println "___________")
    (println "For " ?name " for condition " ?indication " recommend 20ml/kg lr or ns bolus and reassess clinically.\n **lr preferred**")
    (println "___________")
)
"""

env.build(DEFRULE_GREEN_REHYDRATION)

In [17]:
#Defrule for yellow/red/rehydration #4


DEFRULE_YELLOW_RED_REHYDRATION = """
(defrule yellow_red_rehydration "Rule to assess if yellow/red/rehydration criteria met"
    (logical
        (patient (name_is ?name))
        (patient (age_is ?age))
        (supply (supply_status ?supply))
        (fluid (fluid_type ?fluid))
        (indication (fluid_indication ?indication))
  
        (and  
            (indication (fluid_indication rehydration)) ; fluid indication is rehydration  
            (or
                (supply (supply_status yellow)) ; yellow supply status
                (supply (supply_status red)) ; red supply status
            )       
            (or
                (hf (hf_status no))
                (hf (hf_status unknown))
            )   
        )
    )    
    

    => 

    (println "___________")
    (println "For " ?name " for condition " ?indication " recommend 20ml/kg lr or ns bolus and reassess clinically. Consider oral rehydration if possible.\n **lr preferred**")
    (println "___________")
)
"""

env.build(DEFRULE_YELLOW_RED_REHYDRATION)

In [18]:
#Defrule for green/rehydration/hf #5

DEFRULE_GREEN_REHYDRATION_HF = """
(defrule green_rehydration_hf "Rule to assess if green/rehydration/hf criteria met"
    (logical
        (patient (name_is ?name))
        (supply (supply_status ?supply))
        (fluid (fluid_type ?fluid))
        (indication (fluid_indication ?indication))
  
        (and  
            (supply (supply_status green)) ; green supply status
            (indication (fluid_indication rehydration)) ; fluid indication is rehydration  
            (hf (hf_status yes)) ;hf is yes
                              
        )
    )    
    
 =>

   
    (println "___________")
    (println "For " ?name " for condition " ?indication " recommend 15ml/kg lr or ns bolus and reassess clinically.\n **lr preferred and use caution with iv fluids - heart failure patient**")
    (println "___________")
)
"""

env.build(DEFRULE_GREEN_REHYDRATION_HF)

In [19]:
#Defrule for yellow/red/rehydration/hf #6

DEFRULE_YELLOW_RED_REHYDRATION_HF = """
(defrule yellow_red_rehydration_hf "Rule to assess if yellow/red/rehydration/hf criteria met"
    (logical
        (patient (name_is ?name))
        (patient (age_is ?age))
        (supply (supply_status ?supply))
        (fluid (fluid_type ?fluid))
        (indication (fluid_indication ?indication))
  
        (and  
            (indication (fluid_indication rehydration)) ; fluid indication is rehydration          
            (hf (hf_status yes))
            (or
                (supply (supply_status yellow)) ; yellow supply status
                (supply (supply_status red)) ; red supply status)
            )   
        )
    )    
    

    => 

    (println "___________")
    (println "For " ?name " for condition " ?indication " recommend 15ml/kg lr or ns bolus and reassess clinically. Consider oral rehydration if possible.\n **lr preferred and use caution with iv fluids - heart failure patient**")
    (println "___________")
)
"""

env.build(DEFRULE_YELLOW_RED_REHYDRATION_HF)

In [20]:
#Defrule for green/maintenance #7

DEFRULE_GREEN_MAINTENANCE = """
(defrule green_maintenance "Rule to assess if green/maintenance criteria met"
    (logical
        (patient (name_is ?name))
        (supply (supply_status ?supply))
        (fluid (fluid_type ?fluid))
        (indication (fluid_indication ?indication))
  
        (and  
            (supply (supply_status green)) ; green supply status
            (indication (fluid_indication maintenance)) ; fluid indication is maintenance  
            (or
                (hf (hf_status no)) ;hf is no
                (hf (hf_status unknown)) ; hf is unknown
            )                  
        )
    )    
    
 =>

   
    (println "___________")
    (println "For " ?name " for condition " ?indication " recommend 25ml/kg/day lr for up to 1 day and then reassess clinically.")
    (println "___________")
)
"""

env.build(DEFRULE_GREEN_MAINTENANCE)

In [21]:
#Defrule for yellow/maintenance #8
###
DEFRULE_YELLOW_MAINTENANCE = """
(defrule yellow_maintenance "Rule to assess if yellow/maintenance criteria met"
    (logical
        (patient (name_is ?name))
        (patient (age_is ?age))
        (supply (supply_status ?supply))
        (fluid (fluid_type ?fluid))
        (indication (fluid_indication ?indication))
  
        (and  
            (supply (supply_status yellow)) ; yellow supply status
            (indication (fluid_indication maintenance)) ; fluid indication is maintenance          
            (or
                (hf (hf_status no))
                (hf (hf_status unknown))
            )   
        )
    )    
    

    => 

    (println "___________")
    (println "For " ?name " for condition " ?indication " recommend 25ml/kg/day lr for up to 1 day and then reassess clinically. Consider oral hydration if possible.")
    (println "___________")
)
"""

env.build(DEFRULE_YELLOW_MAINTENANCE)

In [22]:
#Defrule for red/maintenance #9
DEFRULE_RED_MAINTENANCE = """
(defrule red_maintenance "Rule to assess if red/maintenance criteria met"
    (logical
        (patient (name_is ?name))
        (supply (supply_status ?supply))
        (fluid (fluid_type ?fluid))
        (indication (fluid_indication ?indication))
  
        (and  
            (supply (supply_status red)) ; red supply status
            (indication (fluid_indication maintenance)) ; fluid indication is maintenance  
            (or
                (hf (hf_status no)) ;hf is no
                (hf (hf_status unknown)) ; hf unknown
            )                 
        )
    )    
    
 =>

   
    (println "___________")
    (println "For " ?name " for condition " ?indication " recommend 20ml/kg/day lr for up to 1 day and then reassess clinically. Consider oral hydration if possible.")
    (println "___________")
)
"""

env.build(DEFRULE_RED_MAINTENANCE)

In [23]:
#Defrule for green/yellow/red/maintenance/hf #10
DEFRULE_GREEN_YELLOW_RED_MAINTENANCE_HF = """
(defrule green_yellow_red_maintenance_hf "Rule to assess if green/yellow/red/maintenance/hf criteria met"
    (logical
        (patient (name_is ?name))
        (patient (age_is ?age))
        (supply (supply_status ?supply))
        (fluid (fluid_type ?fluid))
        (indication (fluid_indication ?indication))
  
        (and  
            (indication (fluid_indication maintenance)) ; fluid indication is maintenance          
            (hf (hf_status yes))
            (or
                (supply (supply_status green)) ; green supply status
                (supply (supply_status yellow)) ; yellow supply status
                (supply (supply_status red)) ; red supply status
            )
        )
    )    
    

    => 

    (println "___________")
    (println "For " ?name " for condition " ?indication " recommend 15ml/kg/day lr for up to 1 day and then reassess clinically.\n **Use caution with iv fluids - heart failure patient**")
    (println "___________")
)
"""

env.build(DEFRULE_GREEN_YELLOW_RED_MAINTENANCE_HF)

In [24]:
#Defrule for green/yellow/contrast #11
DEFRULE_GREEN_YELLOW_CONTRAST = """
(defrule green_yellow_contrast "Rule to assess if green/yellow/contrast criteria met"
    (logical
        (patient (name_is ?name))
        (supply (supply_status ?supply))
        (indication (fluid_indication ?indication))
  
        (and  
            (indication (fluid_indication contrast)) ; fluid indication is contrast  
            (or
                (supply (supply_status green)) ; green supply status
                (supply (supply_status yellow)) ; yellow supply status
            )                  
        )
    )    
    
 =>

   
    (println "___________")
    (println "For " ?name " for condition " ?indication " study recommend 1 ml/kg/hour ns starting 1 hour before contrast study and for 6 hours after contrast study.")
    (println "___________")
)
"""

env.build(DEFRULE_GREEN_YELLOW_CONTRAST)

In [25]:
#Defrule for red contrast #12

DEFRULE_RED_CONTRAST = """
(defrule red_contrast "Rule to assess if red/contrast criteria met"
    (logical
        (patient (name_is ?name))
        (supply (supply_status ?supply))
        (indication (fluid_indication ?indication))
  
        (and  
            (indication (fluid_indication contrast)) ; fluid indication is contrast  
            (supply (supply_status red)) ; red supply status                  
        )
    )    
    
 =>

   
    (println "___________")
    (println "For " ?name " for condition " ?indication " study recommend 0.5ml/kg/hr ns starting 1 hour before contrast study and for 6 hours after contrast study. Consider oral hydration if possible.")
    (println "___________")
)
"""

env.build(DEFRULE_RED_CONTRAST)

In [26]:
#Defrule green/pancreatitis #13
DEFRULE_GREEN_PANCREATITIS = """
(defrule green_pancreatitis "Rule to assess if green/pancreatitis criteria met"
    (logical
        (patient (name_is ?name))
        (supply (supply_status ?supply))
        (fluid (fluid_type ?fluid))
        (indication (fluid_indication ?indication))
  
        (and  
            (indication (fluid_indication pancreatitis)) ; fluid indication is pancreatitis  
            (supply (supply_status green)) ; green supply status
            (or
                (hf (hf_status no)) ; hf no
                (hf (hf_status unknown)) ; hf unknown
            )
        )
    )    
    
 =>

   
    (println "___________")
    (println "For " ?name " for condition " ?indication " recommend 10ml/kg bolus lr or ns then 1.5ml/kg/hr.\n **lr preferred**")
    (println "___________")
)
"""

env.build(DEFRULE_GREEN_PANCREATITIS)

In [27]:
#Defrule yellow/pancreatitis/allcases #14

DEFRULE_YELLOW_PANCREATITIS_ALL = """
(defrule yellow_pancreatitis "Rule to assess if yellow/pancreatitis/all cases criteria met"
    (logical
        (patient (name_is ?name))
        (supply (supply_status ?supply))
        (fluid (fluid_type ?fluid))
        (indication (fluid_indication ?indication))
  
        (and  
            (indication (fluid_indication pancreatitis)) ; fluid indication is pancreatitis  
            (supply (supply_status yellow)) ; yellow supply status
            (or
                (hf (hf_status yes)) ; hf yes
                (hf (hf_status no)) ; hf no
                (hf (hf_status unknown)) ; hf unknown
            )
        )
    )    
    
 =>

   
    (println "___________")
    (println "For " ?name " for condition " ?indication " recommend 10ml/kg bolus lr or ns then 1.5ml/kg/hr and reassess clinically at 6 hours.\n **lr preferred**")
    (println "___________")
)
"""

env.build(DEFRULE_YELLOW_PANCREATITIS_ALL)

In [28]:
#Defule red/pancreatitis/allcases #15
DEFRULE_RED_PANCREATITIS_ALL = """
(defrule red_pancreatitis "Rule to assess if red/pancreatitis/hf or no hf criteria met"
    (logical
        (patient (name_is ?name))
        (supply (supply_status ?supply))
        (fluid (fluid_type ?fluid))
        (indication (fluid_indication ?indication))
  
        (and  
            (indication (fluid_indication pancreatitis)) ; fluid indication is pancreatitis  
            (supply (supply_status red)) ; red supply status
            (or
                (hf (hf_status yes)) ; hf yes
                (hf (hf_status no)) ; hf no
                (hf (hf_status unknown)) ; hf unknown
            )
        )
    )    
    
 =>

   
    (println "___________")
    (println "For " ?name " for condition " ?indication " recommend 5ml/kg bolus lr or ns then 1ml/kg/hr and reassess clinically at 6 hours.\n **lr preferred**")
    (println "___________")
)
"""

env.build(DEFRULE_RED_PANCREATITIS_ALL)

In [29]:
#Defrule green/pancreatitis/hf #16
DEFRULE_GREEN_PANCREATITIS_HF = """
(defrule green_pancreatitis_hf "Rule to assess if green/pancreatitis/hf criteria met"
    (logical
        (patient (name_is ?name))
        (supply (supply_status ?supply))
        (fluid (fluid_type ?fluid))
        (indication (fluid_indication ?indication))
  
        (and  
            (indication (fluid_indication pancreatitis)) ; fluid indication is pancreatitis  
            (hf (hf_status yes)) ; hf yes
            (or
                (supply (supply_status green)) ; green supply status   
            )
        )
    )    
    
 =>

   
    (println "___________")
    (println "For " ?name " for condition " ?indication " recommend 10ml/kg bolus lr or ns then 1.5ml/kg/hr and reassess clinically at 6 hours.\n **lr preferred - use caution with iv fluids - heart failure patient**")
    (println "___________")
)
"""

env.build(DEFRULE_GREEN_PANCREATITIS_HF)

In [30]:
#Defule green/yellow/dka #17
DEFRULE_GREEN_YELLOW_DKA = """
(defrule green_yellow_dka "Rule to assess if green/yellow/dka criteria met"
    (logical
        (patient (name_is ?name))
        (supply (supply_status ?supply))
        (fluid (fluid_type ?fluid))
        (indication (fluid_indication ?indication))
  
        (and  
            (indication (fluid_indication dka)) ; fluid indication is dka  
            (or
                (supply (supply_status green)) ; green supply status   
                (supply (supply_status yellow)) ; yellow supply status
            )
        )
    )    
    
 =>

   
    (println "___________")
    (println "For " ?name " for condition " ?indication " recommend 15-20 mL/kg of body weight per hour lr or ns for the first 1-2 hours, then adjust based on clinical status.\n **lr preferred**")
    (println "___________")
)
"""

env.build(DEFRULE_GREEN_YELLOW_DKA)

In [31]:
#Defule red/dka #18
DEFRULE_RED_DKA = """
(defrule red_dka "Rule to assess if red/dka criteria met"
    (logical
        (patient (name_is ?name))
        (supply (supply_status ?supply))
        (fluid (fluid_type ?fluid))
        (indication (fluid_indication ?indication))
  
        (and  
            (indication (fluid_indication dka)) ; fluid indication is dka  
            (supply (supply_status red)) ; red supply status   
        )
    )    
    
 =>

   
    (println "___________")
    (println "For " ?name " for condition " ?indication " recommend 10-15 ml/kg of body weight per hour lr or ns for the first 1-2 hours, then adjust based on clinical status.\n **lr preferred**")
    (println "___________")
)
"""

env.build(DEFRULE_RED_DKA)

In [33]:
#Execute

env.reset()
env.run()


Enter patient name: Sandy
Enter patient age (in years): 48
What is the indication for IV fluid? (shock rehydration maintenance contrast pancreatitis dka): shock
What is the current IV fluid supply level? (green, yellow, or red): red


Given the indication selected, you may choose a fluid type.


Enter the desired IV fluid type (either lr or ns): lr


___________
For Sandy for condition shock recommend considering vasopressor therapy. Consider 15-30ml/kg lr or ns bolus.
 **lr preferred**
___________


3

In [34]:
print_facts(env)

(hf (hf_status unknown))
(patient (name_is "Sandy") (age_is 48))
(indication (fluid_indication shock))
(supply (supply_status red))
(fluid (fluid_type lr))
Total facts: 5
